# Mini Inception Modeling  with Model Subclassing API

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model

# enable mixed precision
mixed_precision = True
policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
tf.keras.mixed_precision.experimental.set_policy(policy)

# Conv Module 
class ConvModule(tf.keras.layers.Layer):
	def __init__(self, kernel_num, 
                 kernel_size, strides, padding='same'):
		super(ConvModule, self).__init__()
		self.conv = tf.keras.layers.Conv2D(kernel_num, kernel_size=kernel_size, strides=strides, padding=padding)
		self.bn   = tf.keras.layers.BatchNormalization()

	def call(self, input_tensor, training=False):
		x = self.conv(input_tensor)
		x = self.bn(x, training=training)
		x = tf.nn.relu(x)
		return x 

# Inception Module
class InceptionModule(tf.keras.layers.Layer):
    def __init__(self, kernel_size1x1, kernel_size3x3):
        super(InceptionModule, self).__init__()
        self.conv1 = ConvModule(kernel_size1x1, kernel_size=(1,1), strides=(1,1))
        self.conv2 = ConvModule(kernel_size3x3, kernel_size=(3,3), strides=(1,1))
        self.cat   = tf.keras.layers.Concatenate()

    def call(self, input_tensor, training=False):
        x_1x1 = self.conv1(input_tensor)
        x_3x3 = self.conv2(input_tensor)
        x = self.cat([x_1x1, x_3x3])
        return x 

# Downsample Module 
class DownsampleModule(tf.keras.layers.Layer):
	def __init__(self, kernel_size):
		super(DownsampleModule, self).__init__()
		self.conv3 = ConvModule(kernel_size, kernel_size=(3,3), strides=(2,2), padding="valid")
		self.pool  = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2))
		self.cat   = tf.keras.layers.Concatenate()

	def call(self, input_tensor, training=False):
		conv_x = self.conv3(input_tensor, training=training)
		pool_x = self.pool(input_tensor)
		return self.cat([conv_x, pool_x])

# Encompassing Module in Model Class
class MiniInception(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MiniInception, self).__init__()
        self.conv_block = ConvModule(96, (3,3), (1,1))
        self.inception_block1 = InceptionModule(32, 32)
        self.inception_block2 = InceptionModule(32, 48)
        self.downsample_block1 = DownsampleModule(80)
  
        self.inception_block3 = InceptionModule(112, 48)
        self.inception_block4 = InceptionModule(96, 64)
        self.inception_block5 = InceptionModule(80, 80)
        self.inception_block6 = InceptionModule(48, 96)
        self.downsample_block2 = DownsampleModule(96)

        self.inception_block7 = InceptionModule(176, 160)
        self.inception_block8 = InceptionModule(176, 160)

        self.avg_pool = tf.keras.layers.AveragePooling2D((7,7))
        self.flat = tf.keras.layers.Flatten()
        self.classfier = tf.keras.layers.Dense(num_classes, activation='softmax')


    def call(self, input_tensor, training=False,**kwargs):
        x = self.conv_block(input_tensor)
        x = self.inception_block1(x)
        x = self.inception_block2(x)
        x = self.downsample_block1(x)

        x = self.inception_block3(x)
        x = self.inception_block4(x)
        x = self.inception_block5(x)
        x = self.inception_block6(x)
        x = self.downsample_block2(x)

        x = self.inception_block7(x)
        x = self.inception_block8(x)
        x = self.avg_pool(x)

        x = self.flat(x)
        return self.classfier(x)

    def build_graph(self, shape):
        x = tf.keras.layers.Input(shape=shape)
        return Model(inputs=[x], outputs=self.call(x))

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2070 SUPER, compute capability 7.5


## Quick Test

In [2]:
raw_input = (32, 32, 3)

cm = MiniInception()
y  = cm(tf.ones(shape=(0,*raw_input))) # The first call to the `cm` will create the weights

cm.build_graph(raw_input).summary()

# Data Set (CIFAR-10) and Visualization

In [4]:
import numpy as np

import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# train set / data 
x_train = x_train.astype('float32') / 255
# validation set / data 
x_test = x_test.astype('float32') / 255

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

import matplotlib.pyplot as plt 
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[y_train[i][0]])
plt.show()

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [5]:
for i, (x, y) in enumerate(train_dataset):
    print(x.shape, y.shape)
    
    if i == 2:
        break


for i, (x, y) in enumerate(val_dataset):
    print(x.shape, y.shape)
    
    if i == 2:
        break

# Custom Loop Training from Scratch

In [6]:
import time 
epochs = 20
batch_size = 256

# train set / target 
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
# validation set / target 
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.batch(batch_size)

# define model 
model = MiniInception()
model.build_graph().summary()

# Instantiate an optimizer to train the model.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Prepare the metrics.
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric   = tf.keras.metrics.CategoricalAccuracy()

# tensorboard writer 
train_writer = tf.summary.create_file_writer('logs/train/')
test_writer  = tf.summary.create_file_writer('logs/test/')

@tf.function
def train_step(step, x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        train_loss_value = loss_fn(y, logits)
    grads = tape.gradient(train_loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    
    # write training loss and accuracy to the tensorboard
    with train_writer.as_default():
        tf.summary.scalar('loss', train_loss_value, step=step)
        tf.summary.scalar('accuracy', train_acc_metric.result(), step=step) 
    
    return train_loss_value

@tf.function
def test_step(step, x, y):
    val_logits = model(x, training=False)
    # Compute the loss value f
    val_loss_value = loss_fn(y, val_logits)
    # Update val metrics
    val_acc_metric.update_state(y, val_logits)
    
    # write test loss and accuracy to the tensorboard
    with train_writer.as_default():
        tf.summary.scalar('val loss', val_loss_value, step=step)
        tf.summary.scalar('val accuracy', val_acc_metric.result(), step=step) 

    return val_loss_value

# custom training loop 
for epoch in range(epochs):
    t = time.time()
    # batch training 
    # Iterate over the batches of the dataset.
    for train_batch_step, (x_batch_train,\
                           y_batch_train) in enumerate(train_dataset):
        train_batch_step = tf.convert_to_tensor(train_batch_step, dtype=tf.int64)
        train_loss_value = train_step(train_batch_step, x_batch_train, y_batch_train)

    # evaluation on validation set 
    # Run a validation loop at the end of each epoch.
    for test_batch_step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
        test_batch_step = tf.convert_to_tensor(test_batch_step, dtype=tf.int64)
        val_loss_value = test_step(test_batch_step, x_batch_val, y_batch_val)

    template = 'ETA: {} - epoch: {} loss: {}  acc: {} val loss: {} val acc: {}\n'
    print(template.format(
        round((time.time() - t)/60, 2), epoch + 1,
        train_loss_value, float(train_acc_metric.result()),
        val_loss_value, float(val_acc_metric.result())
    ))
        
    # Reset metrics at the end of each epoch
    train_acc_metric.reset_states()
    val_acc_metric.reset_states()

ETA: 0.78 - epoch: 1 loss: 0.7587890625  acc: 0.5794399976730347 val loss: 3.173828125 val acc: 0.10159999877214432

ETA: 0.29 - epoch: 2 loss: 0.63232421875  acc: 0.7421200275421143 val loss: 1.0126953125 val acc: 0.5756999850273132

ETA: 0.32 - epoch: 3 loss: 0.453369140625  acc: 0.8073400259017944 val loss: 0.7734375 val acc: 0.7243000268936157

ETA: 0.33 - epoch: 4 loss: 0.474365234375  acc: 0.8501200079917908 val loss: 0.64111328125 val acc: 0.7628999948501587

ETA: 0.4 - epoch: 5 loss: 0.28369140625  acc: 0.8811399936676025 val loss: 0.65673828125 val acc: 0.6589000225067139

ETA: 0.34 - epoch: 6 loss: 0.288330078125  acc: 0.906059980392456 val loss: 0.794921875 val acc: 0.7311000227928162

ETA: 0.53 - epoch: 7 loss: 0.1297607421875  acc: 0.9277999997138977 val loss: 0.91162109375 val acc: 0.7084000110626221

ETA: 0.31 - epoch: 8 loss: 0.0743408203125  acc: 0.9464399814605713 val loss: 0.5654296875 val acc: 0.7071999907493591

ETA: 0.3 - epoch: 9 loss: 0.0855712890625  acc: 0.957